In [3]:
import tensorflow
print(tensorflow.__version__)

2.18.0


In [4]:
pip uninstall tensorflow -y

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install tensorflow==2.17.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tensorflow
print(tensorflow.__version__)

2.17.0


In [2]:
import keras
print(keras.__version__)

3.8.0


In [3]:
pip uninstall keras -y

Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install keras==3.7.0

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.2 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.2 MB ? eta -:--:--
   ----------------- ---------------------- 0.5/1.2 MB 670.4 kB/s eta 0:00:02
   ----------------- ---------------------- 0.5/1.2 MB 670.4 kB/s eta 0:00:02
   ------------------------- -------------- 0.8/1.2 MB 644.9 kB/s eta 0:00:01
   ------------------------- -------------- 0.8/1.2 MB 644.9 kB/s eta 0:00:01
   ---------------------------------- ----- 1.0/1.2 MB 680.3 kB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 694.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import keras
print(keras.__version__)

3.7.0


In [2]:
from PIL import Image
import face_recognition
import os
from flask import Flask, request, render_template
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from werkzeug.utils import secure_filename
from phone_detection import *
from background_check import *

app = Flask(__name__)
model1= keras.models.load_model("Eye-Detection/eye_detection.h5")
model2= keras.models.load_model("Yawn-Detection/yawn_detection.h5")
model3= keras.models.load_model("Eye-Direction/eye_direction.h5")

@app.route('/')
def home():
    return render_template("index.html")

@app.route('/predict', methods = ["POST", "GET"])
def predict():
    if request.method == "POST":
        f = request.files['img']
        filename = secure_filename(f.filename)
        f.save(os.path.join('/', filename))
        print(os.path.join('/', filename))
        img=image.load_img(os.path.join('/', filename),target_size=(64,64))

        #function to detect cell phone  
        #phone = phone_detection(os.path.join('/', filename))

        #function to check surroundings
        #env = background_check(os.path.join('/', filename))

        #function to detect yawn
        def yawn_detection():
            y=image.img_to_array(img)
            y=np.expand_dims(y,axis=0)
            pred1=np.argmax(model2.predict(y))
            if pred1==0:
                return True
            else:
                return False

        #function to detect eye gaze (open/closed)
        def eyegaze_detection():
            pic = face_recognition.load_image_file(os.path.join('/', filename))
            face_landmarks_list = face_recognition.face_landmarks(pic)
            
            eyes = []
            eyes.append(face_landmarks_list[0]['left_eye'])
            eyes.append(face_landmarks_list[0]['right_eye'])

            for eye in eyes:
                x_max = max([coordinate[0] for coordinate in eye])
                x_min = min([coordinate[0] for coordinate in eye])
                y_max = max([coordinate[1] for coordinate in eye])
                y_min = min([coordinate[1] for coordinate in eye])
                 
            x_range = x_max - x_min
            y_range = y_max - y_min

            if x_range > y_range:
                right = round(.5*x_range) + x_max
                left = x_min - round(.5*x_range)
                bottom = round(((right-left) - y_range))/2 + y_max
                top = y_min - round(((right-left) - y_range))/2
            else:
                bottom = round(.5*y_range) + y_max
                top = y_min - round(.5*y_range)
                right = round(((bottom-top) - x_range))/2 + x_max
                left = x_min - round(((bottom-top) - x_range))/2

            im = Image.open(os.path.join('/', filename))
            im = im.crop((left, top, right, bottom))

            im = im.resize((64,64))
            im.save(os.path.join('/', filename))

            x=image.img_to_array(img)
            x=np.expand_dims(x,axis=0)
            pred=np.argmax(model1.predict(x))

            if pred==0:
                return "closed"
            else:
                return "open"

        def eye_direction():
            x=image.img_to_array(img)
            x=np.expand_dims(x,axis=0)
            pred=np.argmax(model3.predict(x))

            if pred==1:
                return "right"
            else:
                return "left"

        return{
            #"surroundings": env,
            "eyes": eyegaze_detection(),
            "gaze":eye_direction(),
            "yawn": yawn_detection(),
            #"cellphone":phone
        }
       
        
if __name__ == "__main__":
    app.run(debug = False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [3]:
import cv2
import numpy as np
import face_recognition
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from PIL import Image

# Load pre-trained models
model1= keras.models.load_model("Eye-Detection/eye_detection.h5")
model2= keras.models.load_model("Yawn-Detection/yawn_detection.h5")
model3= keras.models.load_model("Eye-Direction/eye_direction.h5")

# Initialize webcam
cap = cv2.VideoCapture(0)  # 0 for default webcam

def preprocess_frame(frame):
    """Prepares a frame for model prediction."""
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return img

def yawn_detection(img):
    """Predicts if the person is yawning."""
    pred = np.argmax(model2.predict(img))
    return "Yawning" if pred == 0 else "Not Yawning"

def eyegaze_detection(frame):
    """Detects if eyes are open or closed using face landmarks."""
    face_landmarks_list = face_recognition.face_landmarks(frame)

    if not face_landmarks_list:
        return "No Face"

    eyes = []
    try:
        eyes.append(face_landmarks_list[0]['left_eye'])
        eyes.append(face_landmarks_list[0]['right_eye'])
    except:
        return "No Eyes Detected"

    # Extract eye coordinates
    for eye in eyes:
        x_max = max([coordinate[0] for coordinate in eye])
        x_min = min([coordinate[0] for coordinate in eye])
        y_max = max([coordinate[1] for coordinate in eye])
        y_min = min([coordinate[1] for coordinate in eye])

    im = Image.fromarray(frame)
    im = im.crop((x_min, y_min, x_max, y_max))
    im = im.resize((64, 64))
    
    x = image.img_to_array(im)
    x = np.expand_dims(x, axis=0)
    pred = np.argmax(model1.predict(x))
    
    return "Closed" if pred == 0 else "Open"

def eye_direction(img):
    """Predicts left or right gaze direction."""
    pred = np.argmax(model3.predict(img))
    return "Looking Right" if pred == 1 else "Looking Left"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img = preprocess_frame(frame)

    # Get predictions
    eye_state = eyegaze_detection(frame)
    yawn_state = yawn_detection(img)
    gaze_direction = eye_direction(img)

    # Display results
    cv2.putText(frame, f"Eyes: {eye_state}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Gaze: {gaze_direction}", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    cv2.putText(frame, f"Yawn: {yawn_state}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Live Prediction", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━